In [1]:
import datasets
import json
import os
import math
import numpy as np
import random

In [2]:
random.seed(42)

In [3]:
# Change these variables to perform evalution for you task
original_quiz_path = os.path.abspath(os.getcwd()).split('user_centric_experiment')[0] + 'processed_data/gpt3/completion_6/processed_test.json'
generated_quiz_path = os.path.abspath(os.getcwd()).split('user_centric_experiment')[0] + 'generated_data_gpt3/experiment_6/generated_quiz.json'

In [4]:
# Get prompts
prompts = []

for line in open(original_quiz_path):
    prompts.append(json.loads(line)['prompt'].split('\n\n###')[0].strip())

# Get original quizzes
original_quizzes = []

for line in open(original_quiz_path):
    original_quizzes.append(json.loads(line)['completion'].split('\n###')[0].strip())
    
# Get generated quizzes
generated_quizzes = []
with open(generated_quiz_path) as f:
    generated_quizzes_dict = json.load(f)

for key in generated_quizzes_dict:
    generated_quizzes.append(generated_quizzes_dict[key].strip())

In [5]:
def shuffle_quizzes(quizzes):
    shuffled_quizzes = []
    
    for i in range(len(quizzes)):
        quiz = quizzes[i].split('\n')
        question = quiz[0]
        answers = quiz[1:]

        random.shuffle(answers)

        shuffled_quiz = question + '\n' + '\n'.join(answers)

        shuffled_quizzes.append(shuffled_quiz)
        
    return shuffled_quizzes

shuffled_original_quizzes = shuffle_quizzes(original_quizzes)
shuffled_generated_quizzes = shuffle_quizzes(generated_quizzes)

In [6]:
human_evaluation_idx = []
excel_idx = np.arange(2,102)

count = 0
for i in range(0,len(original_quizzes),math.floor(len(original_quizzes)/100)):
    human_evaluation_idx.append(i)
    count+=1

    if count == 100:
        break
        
overlap_idx = [9, 36, 117, 234, 270, 324, 369, 432, 459, 477, 549, 585, 594, 666, 684, 711, 729, 747, 792, 846]
overlap_excel = [3, 6, 15, 28, 32, 38, 43, 50, 53, 55, 63, 67, 68, 76, 78, 81, 83, 85, 90, 96]

case_study_idx = [x for x in human_evaluation_idx if x not in overlap_idx]
case_study_excel = [x for x in excel_idx if x not in overlap_excel]

random_idx = random.sample(range(80), 20)

sampled_idx = sorted(random.sample(case_study_idx, 20))
sampled_excel = sorted(random.sample(case_study_excel, 20))

sampled_idx = sorted([case_study_idx[x] for x in random_idx])
sampled_excel = sorted([case_study_excel[x] for x in random_idx])

In [7]:
count = 0
with open('raw_user_centric_experiment.txt', 'w') as f:
    for i in range(len(sampled_idx)):
        f.write('Excel row: ' + str(sampled_excel[i]) + ' Test instance: ' + str(sampled_idx[i]+1) + '\n\n')
        f.write(prompts[sampled_idx[i]] + '\n\n')
        f.write('Original quiz:\n')
        f.write(shuffled_original_quizzes[sampled_idx[i]].strip() + '\n\n')
        f.write('Generated quiz:\n')
        f.write(shuffled_generated_quizzes[sampled_idx[i]].strip() + '\n\n')
        f.write('----------------------------------------------------------------------------------------' + '\n\n')
        count+=1
        
        if count == 100:
            break